In [1]:
#CPU AVerage Load

import os
import subprocess
import re

statistics = {}


physical_and_logical_cpu_count = os.cpu_count()
statistics['physical_and_logical_cpu_count'] = physical_and_logical_cpu_count


cpu_load = [x / os.cpu_count() * 100 for x in os.getloadavg()][-1]
statistics['cpu_load'] = cpu_load
print(statistics['cpu_load'])

196.0


In [2]:
#Available Memory Percentage

import psutil


print(psutil.virtual_memory().percent)
print(psutil.virtual_memory().available * 100 / psutil.virtual_memory().total)

70.1
29.9508201272313


In [3]:
#CPU Percentage Utilisation

psutil.cpu_percent()

61.3

In [4]:
#Disk Usage Percentage

import os
import psutil

psutil.disk_usage(os.sep).percent

43.2

In [5]:
# Network latency

ping_result = subprocess.run(['ping', '-i 5', '-c 5', 'google.com'], stdout=subprocess.PIPE).stdout.decode('utf-8').split('\n')

min, avg, max = ping_result[-2].split('=')[-1].split('/')[:3]
statistics['network_latency'] = dict(
    {
        'min': min.strip(),
        'avg': avg.strip(),
        'max': max.strip(),
    }
)

print(statistics['network_latency'])

{'min': '41.515', 'avg': '45.353', 'max': '57.465'}


In [6]:
#Send and Get Data to and from ElasticSearch DataBase

import elasticsearch
from elasticsearch import Elasticsearch
import os
import json
import psutil

def send_data_to_es(data):
    es = Elasticsearch(['localhost:9200'])
    res = es.index(index='metrics',doc_type='metrics_data',body=data)
    print(res)
    
def get_data_from_es():
    es = Elasticsearch(['localhost:9200'])
    r = es.search(index="metrics",body={"query": {"match":{'Name':'Performance_Metrics'}}})
    print(r)
    print(type(r))
    print(r["hits"]["hits"][0]["_source"])
    
data = {"Name":"Performance_Metrics","CPU-Load":cpu_load,"Virtual-Memory-Percent":psutil.virtual_memory().percent,"Disk-Usage-Percent":psutil.disk_usage(os.sep).percent}

#send_data_to_es(data)
get_data_from_es()

{'took': 222, 'timed_out': False, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}, 'hits': {'total': 1, 'max_score': 0.2876821, 'hits': [{'_index': 'metrics', '_type': 'metrics_data', '_id': '7yrfZnoBpmYLlLWSv9NX', '_score': 0.2876821, '_source': {'Name': 'Performance_Metrics', 'CPU-Load': 163.0, 'Virtual-Memory-Percent': 93.7, 'Disk-Usage-Percent': 43.0}}]}}
<class 'dict'>
{'Name': 'Performance_Metrics', 'CPU-Load': 163.0, 'Virtual-Memory-Percent': 93.7, 'Disk-Usage-Percent': 43.0}
